
# Retail Analytics 101 with PySpark (Databricks Free)

This notebook teaches PySpark basics on Databricks using a small **retail** case study.
You will:
- Load CSVs from DBFS
- Inspect and clean data (nulls, duplicates, outliers)
- Parse JSON, join dimensions, derive metrics
- Aggregate with groupBy and window functions
- Save curated tables as Delta

> **Before you start:** upload the CSVs from `retail_case_study_csv.zip` into **Workspace ➜ Files** or `/FileStore/retail_case_study` on Databricks.


In [0]:
# ✅ Databricks FREE-safe CSV loader (no DBFS, no local file IO by Spark)
# - Uses pandas to fetch CSV from a RAW GitHub URL into memory
# - Converts to Spark DataFrame (Spark never touches the filesystem)
import pandas as pd
from pyspark.sql import functions as F

def load_csv_from_github(raw_url: str):
    pdf = pd.read_csv(raw_url)
    return spark.createDataFrame(pdf)

# Example raw URLs for your repo (edit as needed):
customers_raw = "https://raw.githubusercontent.com/Naviden/Big-Data-for-students/main/Data/customers.csv"
transactions_raw = "https://raw.githubusercontent.com/Naviden/Big-Data-for-students/main/Data/transactions.csv"
products_raw = "https://raw.githubusercontent.com/Naviden/Big-Data-for-students/main/Data/products.csv"

# Example usage:
customers = load_csv_from_github(customers_raw)
transactions = load_csv_from_github(transactions_raw)
products = load_csv_from_github(products_raw)

In [0]:

# If running on Databricks, Spark is already available as 'spark'
from pyspark.sql import functions as F, types as T
from pyspark.sql.window import Window

# Adjust this base path if you uploaded elsewhere
BASE = "/dbfs/FileStore/retail_case_study"

displayHTML(f"<b>Reading from:</b> {BASE}")



## (Optional) Unzip helper
If you uploaded `retail_case_study_csv.zip` into `/FileStore`, run this to extract CSVs into `/FileStore/retail_case_study`.


In [0]:

import os, zipfile

zip_path = "/dbfs/FileStore/retail_case_study_csv.zip"   # change if needed
target_dir = "/dbfs/FileStore/retail_case_study"

if os.path.exists(zip_path):
    os.makedirs(target_dir, exist_ok=True)
    with zipfile.ZipFile(zip_path, "r") as zf:
        zf.extractall(target_dir)
    print("Unzipped to", target_dir)
else:
    print("ZIP not found at", zip_path, "- skip if you already uploaded CSVs to", target_dir)



## Load data
We prefer CSV for simplicity. (Excel requires the `spark-excel` connector on the cluster.)


In [0]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window

def load_csv_from_github(raw_url: str):
    pdf = pd.read_csv(raw_url)
    return spark.createDataFrame(pdf)

In [0]:
customers = load_csv_from_github("https://raw.githubusercontent.com/Naviden/Big-Data-for-students/main/Data/customers.csv")
products  = load_csv_from_github("https://raw.githubusercontent.com/Naviden/Big-Data-for-students/main/Data/products.csv")
stores    = load_csv_from_github("https://raw.githubusercontent.com/Naviden/Big-Data-for-students/main/Data/stores.csv")
tx_raw    = load_csv_from_github("https://raw.githubusercontent.com/Naviden/Big-Data-for-students/main/Data/transactions.csv")
returns   = load_csv_from_github("https://raw.githubusercontent.com/Naviden/Big-Data-for-students/main/Data/returns.csv")

In [0]:


print("Rows -> customers:", customers.count(), "products:", products.count(), "stores:", stores.count(), "transactions:", tx_raw.count(), "returns:", returns.count())


Rows -> customers: 250 products: 60 stores: 12 transactions: 3512 returns: 94



## Inspect schema and some rows


In [0]:

customers.printSchema()
products.printSchema()
stores.printSchema()
tx_raw.printSchema()

display(tx_raw.limit(5))


root
 |-- customer_id: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- signup_date: string (nullable = true)
 |-- age: long (nullable = true)
 |-- loyalty_tier: string (nullable = true)
 |-- is_student: long (nullable = true)
 |-- home_city: string (nullable = true)
 |-- home_region: string (nullable = true)

root
 |-- product_id: long (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- unit_price_list: double (nullable = true)
 |-- tax_rate: double (nullable = true)

root
 |-- store_id: long (nullable = true)
 |-- store_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- region: string (nullable = true)
 |-- opened_date: string (nullable = true)
 |-- store_type: string (nullable = true)

root
 |-- transaction_id: long (nullable = true)
 |-- transaction_ts: string (nullable = t

transaction_id,transaction_ts,store_id,customer_id,product_id,quantity,unit_price,discount_pct,payment_method,channel,review_rating,promo_meta_json,is_returned,pre_tax_amount,tax_rate,tax_amount,total_amount
1,2024-11-13 10:02:00,4,147,9,1,2.75,0.0,Cash,Online,null,"{""coupon"": ""NONE"", ""bundle"": false, ""campaign"": ""BlackFriday""}",0,2.75,0.1,0.28,3.03
2,2025-01-30 19:00:00,1,138,15,2,32.66,0.0,Card,InStore,3.0,"{""coupon"": ""LOYAL5"", ""bundle"": false, ""campaign"": ""BlackFriday""}",0,65.32,0.07,4.57,69.89
3,2024-10-18 18:46:00,6,208,54,2,23.82,0.1,GiftCard,ClickAndCollect,4.0,"{""coupon"": ""WEEKEND15"", ""bundle"": true, ""campaign"": ""SpringSale""}",0,42.88,0.07,3.0,45.88
4,2024-01-16 18:29:00,1,244,57,3,29.88,0.2,Card,InStore,4.0,"{""coupon"": ""LOYAL5"", ""bundle"": false, ""campaign"": ""SpringSale""}",0,71.71,0.0,0.0,71.71
5,2024-12-24 12:13:00,8,145,29,2,41.9,0.0,MobilePay,InStore,4.0,"{""coupon"": ""NONE"", ""bundle"": false, ""campaign"": ""SpringSale""}",0,83.8,0.1,8.38,92.18



## Cast types and parse JSON column


In [0]:

tx = (tx_raw
  .withColumn("transaction_id", F.col("transaction_id").cast("long"))
  .withColumn("store_id",       F.col("store_id").cast("int"))
  .withColumn("customer_id",    F.col("customer_id").cast("int"))
  .withColumn("product_id",     F.col("product_id").cast("int"))
  .withColumn("quantity",       F.col("quantity").cast("int"))
  .withColumn("unit_price",     F.col("unit_price").cast("double"))
  .withColumn("discount_pct",   F.col("discount_pct").cast("double"))
  .withColumn("review_rating",  F.col("review_rating").cast("double"))
  .withColumn("is_returned",    F.col("is_returned").cast("int"))
  .withColumn("pre_tax_amount", F.col("pre_tax_amount").cast("double"))
  .withColumn("tax_rate",       F.col("tax_rate").cast("double"))
  .withColumn("tax_amount",     F.col("tax_amount").cast("double"))
  .withColumn("total_amount",   F.col("total_amount").cast("double"))
  .withColumn("transaction_ts", F.to_timestamp("transaction_ts"))
)

promo_schema = T.StructType([
  T.StructField("coupon",   T.StringType()),
  T.StructField("bundle",   T.BooleanType()),
  T.StructField("campaign", T.StringType())
])

tx = tx.withColumn("promo_meta", F.from_json("promo_meta_json", promo_schema))

display(tx.limit(5))


transaction_id,transaction_ts,store_id,customer_id,product_id,quantity,unit_price,discount_pct,payment_method,channel,review_rating,promo_meta_json,is_returned,pre_tax_amount,tax_rate,tax_amount,total_amount,promo_meta
1,2024-11-13T10:02:00.000Z,4,147,9,1,2.75,0.0,Cash,Online,null,"{""coupon"": ""NONE"", ""bundle"": false, ""campaign"": ""BlackFriday""}",0,2.75,0.1,0.28,3.03,"List(NONE, false, BlackFriday)"
2,2025-01-30T19:00:00.000Z,1,138,15,2,32.66,0.0,Card,InStore,3.0,"{""coupon"": ""LOYAL5"", ""bundle"": false, ""campaign"": ""BlackFriday""}",0,65.32,0.07,4.57,69.89,"List(LOYAL5, false, BlackFriday)"
3,2024-10-18T18:46:00.000Z,6,208,54,2,23.82,0.1,GiftCard,ClickAndCollect,4.0,"{""coupon"": ""WEEKEND15"", ""bundle"": true, ""campaign"": ""SpringSale""}",0,42.88,0.07,3.0,45.88,"List(WEEKEND15, true, SpringSale)"
4,2024-01-16T18:29:00.000Z,1,244,57,3,29.88,0.2,Card,InStore,4.0,"{""coupon"": ""LOYAL5"", ""bundle"": false, ""campaign"": ""SpringSale""}",0,71.71,0.0,0.0,71.71,"List(LOYAL5, false, SpringSale)"
5,2024-12-24T12:13:00.000Z,8,145,29,2,41.9,0.0,MobilePay,InStore,4.0,"{""coupon"": ""NONE"", ""bundle"": false, ""campaign"": ""SpringSale""}",0,83.8,0.1,8.38,92.18,"List(NONE, false, SpringSale)"



## Remove duplicate lines and flag outliers


In [0]:

w = Window.partitionBy("transaction_id", "product_id").orderBy(F.col("transaction_ts").desc_nulls_last())
deduped = tx.withColumn("rn", F.row_number().over(w)).filter("rn = 1").drop("rn")

# Flag implausible quantities (keep them but mark for QA)
cleaned = deduped.withColumn("is_outlier_qty", F.col("quantity") > 20)

print("Original tx rows:", tx_raw.count(), " | After de-dup:", deduped.count())
display(cleaned.filter("is_outlier_qty = true").select("transaction_id","product_id","quantity","total_amount").limit(10))


Original tx rows: 3512  | After de-dup: 3500


transaction_id,product_id,quantity,total_amount
1486,52,250,2120.0
1642,1,100,4607.0
2188,56,100,4841.0
2290,11,100,156.0
2780,30,100,1570.0



## Join dimensions and compute derived metrics


In [0]:

products_t = products.select(
    F.col("product_id").cast("int").alias("product_id"),
    "product_name","category","brand","unit_price_list","tax_rate"
)

customers_t = customers.select(
    F.col("customer_id").cast("int").alias("customer_id"),
    "first_name","last_name","email","signup_date","age","loyalty_tier","is_student","home_city","home_region"
)

stores_t = stores.select(
    F.col("store_id").cast("int").alias("store_id"),
    "store_name","city","region","opened_date","store_type"
)

fact = (cleaned
    .join(products_t, "product_id", "left")
    .join(customers_t, "customer_id", "left")
    .join(stores_t, "store_id", "left")
    .withColumn("year", F.year("transaction_ts"))
    .withColumn("month", F.month("transaction_ts"))
    .withColumn("net_after_discount", (F.col("quantity")*F.col("unit_price")*(1-F.col("discount_pct"))).cast("double"))
)

display(fact.limit(5))


store_id,customer_id,product_id,transaction_id,transaction_ts,quantity,unit_price,discount_pct,payment_method,channel,review_rating,promo_meta_json,is_returned,pre_tax_amount,tax_rate,tax_amount,total_amount,promo_meta,is_outlier_qty,product_name,category,brand,unit_price_list,tax_rate,first_name,last_name,email,signup_date,age,loyalty_tier,is_student,home_city,home_region,store_name,city,region,opened_date,store_type,year,month,net_after_discount
4,147,9,1,2024-11-13T10:02:00.000Z,1,2.75,0.0,Cash,Online,null,"{""coupon"": ""NONE"", ""bundle"": false, ""campaign"": ""BlackFriday""}",0,2.75,0.1,0.28,3.03,"List(NONE, false, BlackFriday)",false,Bakery Item 9,Bakery,Soylent,2.57,0.1,Mackenzie,Oconnell,jfrench@gmail.com,2023-08-11,70,Bronze,1,North Scott,South Dakota,Store #4,Taylorland,Arizona,2022-02-11,Urban,2024,11,2.75
1,138,15,2,2025-01-30T19:00:00.000Z,2,32.66,0.0,Card,InStore,3.0,"{""coupon"": ""LOYAL5"", ""bundle"": false, ""campaign"": ""BlackFriday""}",0,65.32,0.07,4.57,69.89,"List(LOYAL5, false, BlackFriday)",false,Produce Item 15,Produce,Acme,31.2,0.07,Tara,Haas,dorothyholmes@yahoo.com,2023-09-04,44,Gold,0,Port Miranda,Ohio,Store #1,Salinasshire,Georgia,2021-08-09,Suburban,2025,1,65.32
6,208,54,3,2024-10-18T18:46:00.000Z,2,23.82,0.1,GiftCard,ClickAndCollect,4.0,"{""coupon"": ""WEEKEND15"", ""bundle"": true, ""campaign"": ""SpringSale""}",0,42.88,0.07,3.0,45.88,"List(WEEKEND15, true, SpringSale)",false,Dairy Item 54,Dairy,Initech,21.11,0.07,Robert,Nichols,kmartin@yahoo.com,2025-03-11,34,Bronze,0,Lake Bryanhaven,Hawaii,Store #6,Villarrealberg,Massachusetts,2021-08-26,Urban,2024,10,42.876000000000005
1,244,57,4,2024-01-16T18:29:00.000Z,3,29.88,0.2,Card,InStore,4.0,"{""coupon"": ""LOYAL5"", ""bundle"": false, ""campaign"": ""SpringSale""}",0,71.71,0.0,0.0,71.71,"List(LOYAL5, false, SpringSale)",false,Bakery Item 57,Bakery,Umbrella,27.95,0.0,James,Henderson,cooperjessica@gmail.com,2022-08-19,73,Bronze,0,South David,Connecticut,Store #1,Salinasshire,Georgia,2021-08-09,Suburban,2024,1,71.712
8,145,29,5,2024-12-24T12:13:00.000Z,2,41.9,0.0,MobilePay,InStore,4.0,"{""coupon"": ""NONE"", ""bundle"": false, ""campaign"": ""SpringSale""}",0,83.8,0.1,8.38,92.18,"List(NONE, false, SpringSale)",false,Beverages Item 29,Beverages,Stark,41.45,0.1,Nicole,Daniels,ksweeney@davis-phillips.com,2022-12-21,46,Gold,0,Jenniferstad,Tennessee,Store #8,North Carla,Indiana,2022-11-15,Suburban,2024,12,83.8



## Aggregations


In [0]:

rev_month_cat = (fact
  .groupBy("year","month","category")
  .agg(F.round(F.sum("total_amount"),2).alias("revenue"))
  .orderBy("year","month","category"))

top_products = (fact
  .groupBy("product_id","product_name")
  .agg(F.round(F.sum("total_amount"),2).alias("revenue"))
  .orderBy(F.desc("revenue"))
  .limit(10))

display(rev_month_cat)
display(top_products)


year,month,category,revenue
2024,1,Bakery,1511.8
2024,1,Beverages,1394.89
2024,1,Dairy,384.07
2024,1,Household,1745.28
2024,1,Personal Care,1045.76
2024,1,Produce,1373.09
2024,1,Snacks,691.17
2024,2,Bakery,1722.42
2024,2,Beverages,2387.83
2024,2,Dairy,628.83


product_id,product_name,revenue
56,Bakery Item 56,10876.69
1,Beverages Item 1,8824.54
28,Snacks Item 28,7696.03
5,Bakery Item 5,7540.5
17,Bakery Item 17,7529.04
10,Personal Care Item 10,6860.11
45,Produce Item 45,6134.35
7,Produce Item 7,6052.37
50,Household Item 50,5817.89
36,Beverages Item 36,5779.0



## Window functions: rank largest transactions per store


In [0]:

w_store = Window.partitionBy("store_id").orderBy(F.desc("total_amount"))
ranked = fact.select("store_id","transaction_id","total_amount")              .withColumn("rank_in_store", F.rank().over(w_store))

display(ranked.filter("rank_in_store <= 3").orderBy("store_id","rank_in_store"))


store_id,transaction_id,total_amount,rank_in_store
1,235,315.44,1
1,468,290.33,2
1,1328,276.06,3
2,3319,298.36,1
2,2071,272.9,2
2,2175,237.15,3
3,2188,4841.0,1
3,2780,1570.0,2
3,71,327.58,3
4,2099,346.77,1



## UDF for price buckets and JSON campaign analysis


In [0]:

from pyspark.sql.types import StringType

@F.udf(StringType())
def price_bucket(x):
    if x is None: return "unknown"
    return "low" if x < 10 else ("mid" if x < 30 else "high")

with_bucket = fact.withColumn("price_bucket", price_bucket("unit_price"))
display(with_bucket.groupBy("price_bucket").count())

campaign_perf = (fact
  .withColumn("campaign", F.col("promo_meta.campaign"))
  .groupBy("campaign").agg(F.count("*").alias("tx"), F.round(F.sum("total_amount"),2).alias("revenue"))
  .orderBy(F.desc("revenue")))

display(campaign_perf)


price_bucket,count
mid,1083
low,685
high,1732


campaign,tx,revenue
BackToSchool,894,56831.78
None,887,56120.14
SpringSale,873,52165.78
BlackFriday,846,51400.41
